# Problem Formulation

## State Description

An environment dependent on it's own size where only one action is relevant: to clean out the grid so that every tile is 'off'. In order to do this, the program goes through every single tile and decides which tiles to turn according to a set pattern described in the transition model. 


Board size NxN
Cleaner 

## Inital State

Board(SIZE)

## Actions

* TAB

## Transition model

| Operator | Pre-conditions | Changes |
| -------- | -------------- | ------- |
| `tab(row, col)` | (row - 1, col) == true OR <br> (row, col - 1) == true OR <br> (row, col + 1) == true OR <br> (row + 1, col) == true | toggle value at (row - 1, col) AND <br> toggle value at (row, col - 1) AND <br> toggle value at (row, col + 1) AND <br> toggle value at (row + 1, col) |

## Goal State

```
sub allCellsAreEmpty(board)
	for row in board
		for cell in row
			if cell is true
				return false
	return true
```

## Path Cost

Because of the nature of the problem, the only measurable action is 'Tab'; this means that there will be a counter that will go up 1 point for everytime it turns a tile within the grid. There will be no counter for the amount of movements the program does when viewing the grid.

* TAB = +1

# Clean Up Puzzle Implementation

## Dependencies

* `search.py` from the aima proyect
* `utils.py` from the aima proyect

## Board Utils

In [5]:
import random

# getAffectedCellsByPosition
# retrieves all affected cells based on the given position
# @param {Tuple} position
# @param {Integer} size
# @return {List<Tuple>}
def getAffectedCellsByPosition(position, size):
    row, col = position
    allAffectedCells = []
    # order matters
    if row - 1 >= 0: allAffectedCells.append((row - 1, col))
    if col - 1 >= 0: allAffectedCells.append((row, col - 1))
    if col + 1 < size: allAffectedCells.append((row, col + 1))
    if row + 1 < size: allAffectedCells.append((row + 1, col))

    return allAffectedCells

# buildGrid
# creates a string representation of a matrix based on the given size
# if filler is provided, tiles are not placed randomly
# filler is meant to create specific matrix configurations
# @param {Integer} size (default: 5)
# @param {Integer} maxTiles (default: 10)
# @param {Function} filler (default: None)
# @return {String}
def buildGrid(size = 5, maxTiles = 10, filler = None):
    tileCounter = 0
    rows = list()
    for r in range(size):
        row = list()
        for c in range(size):
            if filler:
                row.append(filler(r, c))
            else:
                strBit = '0'
                if tileCounter < maxTiles:
                    bit = random.randint(0, 1)
                    if (bit > 0):
                        tileCounter += 1
                        strBit = str(bit)
                row.append(strBit)
        rows.append('|'.join(row))
    return '\n'.join(rows)

# toggleCell
# toggles the cell's value based on a given location in the given encoded board
# @param {String} board
# @param {Tuple} position
# @return {String}
def toggleCell(board, position):
    rows = board.split('\n')
    affectedCells = getAffectedCellsByPosition(position, len(rows))
    for cell in affectedCells:
        row = rows[cell[0]].split('|')
        row[cell[1]] = '1' if row[cell[1]] == '0' else '0'
        rows[cell[0]] = '|'.join(row)
    return '\n'.join(rows)


# parseBoard
# parses the encoded board into a List<List<String>>
# @param {String} board
# @return {List<List<String>>}
def parseBoard(board):
    rows = []
    encodedRows = board.split('\n')
    for encodedRow in encodedRows:
        rows.append(encodedRow.split('|'))
    return rows

## Display util

In [7]:
def displayProblemSolution(goal_node, methodName, processingTime):
    print('METHOD: ' + methodName)
    print('--- search terminated after %s seconds ---' % (processingTime))
    if not goal_node:
        print('has not found a solution')
        return
    actions = goal_node.solution()
    nodes = goal_node.path()
    print('SOLUTION')
    print('State:' )
    print(nodes[0].state)
    for na in range(len(actions)):
        print('ACTION: ' + str(actions[na]))
        print('State:')
        print(nodes[na+1].state)
        print('END')

## CleanUpPuzzle class

In [8]:
from search import Problem

#### allow actions
TAB = 'TAB'

class CleanUpPuzzle(Problem):
    def __init__(self, initial, goal):
        Problem.__init__(self, initial, goal)
       
    # actions
    # computes all possible movements
    # that can be perfomed on the state
    def actions(self, state):
        return getPossibleMovements(state)

    # result
    # changes the state based on a given action
    def result(self, state, action):
        return toggleCell(state, action[1])

# getPossibleMovements 
# scans the full board to retrieve all tabbable cells
def getPossibleMovements(state):
    board = parseBoard(state)
    size = len(board)
    actions = []
    for row in range(size):
        for col in range(size):
            affectedCells = getAffectedCellsByPosition((row, col), size)
            for cell in affectedCells:
                if board[cell[0]][cell[1]] == '1':
                    actions.append((TAB, (row, col)))
                    break
    return actions

## Testing

### Easy Board Configuration

In [9]:
import time
from search import depth_limited_search
from search import breadth_first_search
from search import uniform_cost_search

BOARD_SIZE = 6

def easyBoard(row, col): 
    if row == 0 and col == 1: return '1'
    if row == 0 and col == 4: return '1'
    if row == 1 and col == 0: return '1'
    if row == 1 and col == 2: return '1'
    if row == 1 and col == 5: return '1'
    if row == 2 and col == 1: return '1'
    if row == 3 and col == 3: return '1'
    if row == 4 and col == 2: return '1'
    if row == 4 and col == 4: return '1'
    if row == 5 and col == 3: return '1'

    return '0'

start_time = time.time()
easy_goal = depth_limited_search(
    CleanUpPuzzle(
        buildGrid(size = BOARD_SIZE, filler = easyBoard), 
        buildGrid(size = BOARD_SIZE)
    ), 
    limit=5
)

displayProblemSolution(easy_goal, 'depth_limited_search', time.time() - start_time)

start_time = time.time()
easy_goal = breadth_first_search(
    CleanUpPuzzle(
        buildGrid(size = BOARD_SIZE, filler = easyBoard), 
        buildGrid(size = BOARD_SIZE)
    )
)
print(easy_goal)
displayProblemSolution(easy_goal, 'breadth_first_search', time.time() - start_time)

start_time = time.time()
easy_goal = uniform_cost_search(
    CleanUpPuzzle(
        buildGrid(size = BOARD_SIZE, filler = easyBoard), 
        buildGrid(size = BOARD_SIZE)
    )
)
displayProblemSolution(easy_goal, 'uniform_cost_search', time.time() - start_time)

METHOD: depth_limited_search
--- search terminated after 18.40137004852295 seconds ---


AttributeError: 'str' object has no attribute 'solution'

### Hard Board Configation

*Note*
the methods depth_limited_search, and uniform_cost_search cost take forever to compute this board and for that reason they will be excluded




In [ ]:
HARD_BOARD = (
    '0|1|0|0|0|1',
    '0|0|1|0|0|0',
    '0|0|0|1|0|1',
    '0|1|0|0|1|0',
    '0|0|1|0|0|0',
    '0|0|0|0|0|0'
)

start_time = time.time()
easy_goal = breadth_first_search(
    CleanUpPuzzle(
        '\n'.join(HARD_BOARD), 
        buildGrid(size = BOARD_SIZE)
    )
)
displayProblemSolution(easy_goal, 'breadth_first_search', time.time() - start_time)


### Could they solve all or some problems? 

The chosen algorithms could solve all solvable problems. Whenever the set problem left a single a tile on, it would try to solve it even though it had no solution. Additionally, it is clear that the algorithms were heavily affected by the amount of cells that were on at the beginning rather than by the size of the actual grid. These are the reasons why the decision of setting the problems randomly was not made, and why there are set values for the selected grids.

### How efficient were they? 

The PSA using BFS is very inefficient because it considers all possible paths for each node. That means that everytime we look for all possible and coherent moves (in an NxN complexity), we exhaustibly search all possible paths for the current known possible paths, adding an exponential complexity (x^L being x the amount of possible paths and L the amount of "clicks" or levels to solve the problem optimally). At the end we'll have the complexity as follows: x^L * N * N. In the scenario were we have 15 possible paths in a problem with 5 steps as an optimal solution with a grid of 8x8 we'll have an approximate search of 15*15*15*15*15*8*8 = 48600000 nodes. However BFS performs betters than DFS and UCS

In the case of DFS, this performs poorly in for the most simple solution. The only way increase its performance is by limiting how depth it can search.

Lastly, UCS's performance is similar for simple and medium problems.


### Was it difficult to program the PSA? 

The PSA implementation was straight forward since most of the heaving lifting was handled by search methods and base class `Problem`

### Also, which algorithm seems to be the best for the selected problems?

After testing the following methods:

* Depth first search

this seems to be the worst search algorithm to solve `CleanUpPuzzle` problem. there were escenarios where this method got stuck and it kept searching for a solution endlessly. DFS should be used for board configurations that are easy to be solved.

* Breadth first search

this seems to be the best algorithm because it was always able to find a solution consistently for different solvable board configurations.

* Uniformed cost search

UCS is more efficient than DFS but it is less efficient compared to BSF.

